In [32]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from IPython.display import display, Markdown
llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

load_dotenv()

True

In [33]:
prompt = """who won the men's ICC T20 Cricket championships in 2024?. Do not make up answers, only answer if you are sure. 
If the timeline is beyond your trained timeline, let me know explicitly as well."""

# it will either not answer or end up hallucinating the answer
response = llm.invoke(prompt)
display(Markdown(response.content))

I do not have information on events that occurred after October 2023, including the winner of the men's ICC T20 Cricket championships in 2024. Please check the latest sources for the most current information.

In [34]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.tools import tool
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [35]:
@tool
def search_web(query: str) -> list:
    """Search the web for a query."""
    print('Searching web and extracting page info')
    wrapper = DuckDuckGoSearchAPIWrapper(max_results=5)
    results = wrapper.results(query, max_results=5)
    docs = []
    for result in tqdm(results):
        # Sending a request to the URL
        response = requests.get(result['link'])
        # Parsing the page content
        soup = BeautifulSoup(response.content, 'html.parser')
        # Extracting all text content from the page
        text_content = soup.get_text(separator="\n", strip=True)
        docs.append(text_content)
    return docs

In [40]:
r = search_web("What is Langgraph?")

Searching web and extracting page info


100%|██████████| 5/5 [00:01<00:00,  3.23it/s]


In [41]:
print(r)

['Just a moment...\nEnable JavaScript and cookies to continue', 'Mastering LangGraph: A Beginner\'s Guide to Building Intelligent Language Models | Medium\nOpen in app\nSign up\nSign in\nWrite\nSign up\nSign in\nIntroduction to LangGraph: A Beginner’s Guide\nCPlog\n·\nFollow\n3 min read\n·\nFeb 15, 2024\n--\n9\nListen\nShare\nLangGraph is a powerful tool for building stateful, multi-actor applications with Large Language Models (LLMs). It extends the LangChain library, allowing you to coordinate multiple chains (or actors) across multiple steps of computation in a cyclic manner. In this article, we’ll introduce LangGraph, walk you through its basic concepts, and share some insights and common points of confusion for beginners.\nWhat is LangGraph?\nLangGraph is a library built on top of LangChain, designed to add cyclic computational capabilities to your LLM applications. While LangChain allows you to define chains of computation (Directed Acyclic Graphs or DAGs), LangGraph introduces t

In [46]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.tools import tool
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

@tool
def search_web(query: str) -> list:
    """Search the web for a query."""
    print('Searching web and extracting page info')
    wrapper = DuckDuckGoSearchAPIWrapper(max_results=5)
    results = wrapper.results(query, max_results=5)
    docs = []
    for result in tqdm(results):
        # Sending a request to the URL
        response = requests.get(result['link'])
        # Parsing the page content
        soup = BeautifulSoup(response.content, 'html.parser')
        # Extracting all text content from the page
        text_content = soup.get_text(separator="\n", strip=True)
        docs.append(text_content)
    return docs

In [47]:
chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web]
chatgpt_with_tools = chatgpt.bind_tools(tools)
prompt = "Who won the most recent Australian Open Tennis Championships in the Men's and Ladies Singles? If the time line is not in sync with your training timeline, be explicit about it."
response = chatgpt_with_tools.invoke(prompt)


response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mSnmzNiqzwVPGiuwVbl9E0A8', 'function': {'arguments': '{"query": "most recent Australian Open Men\'s Singles winner"}', 'name': 'search_web'}, 'type': 'function'}, {'id': 'call_YLGk90fGRLDU9cFMoqHdayg0', 'function': {'arguments': '{"query": "most recent Australian Open Ladies Singles winner"}', 'name': 'search_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 78, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5ba974da-0f20-4639-a039-a860a4efa224-0', tool_calls=[{'name': 'search_web', 'args': {'query': "most recent Australian Open Men's Sing

In [26]:
response.tool_calls

[{'name': 'search_web',
  'args': {'query': "most recent Australian Open Men's Singles winner"},
  'id': 'call_Kdes7xgdjTtDOnw1jB9oJBgQ',
  'type': 'tool_call'},
 {'name': 'search_web',
  'args': {'query': 'most recent Australian Open Ladies Singles winner'},
  'id': 'call_YsahCyzSOJxccT7AK7dAG6i1',
  'type': 'tool_call'}]

In [48]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYS_PROMPT = """You run in a loop of Thought, Action, PAUSE, Observation.
                At the end of the loop, you output an Answer.
                Use Thought to describe your thoughts about the question you have been asked.
                Use Action to run one of the actions available to you - then return PAUSE.
                Observation will be the result of running those actions.
                For the user query below, first break it into multiple sub queries or sub steps if needed.
                The follow the below workflow for each sub query or sub step
                Analyze, validate the relevant results and then compile to generate the final answer

                Use the following workflow format:
                Question: the input task you must solve
                Thought: you should always think about what to do
                Action: the action to take:
                          - should be either to break down the task into sub tasks first if needed
                          - refer to your trained knowledge or call the tool [search_web]
                            if you need to get additional information from the web
                Action Input: the input to the action
                Observation: the result of the action
                ... (this Thought/Action/Action Input/Observation can repeat N times)
                Thought: I now know the final answer
                Final Answer: the final answer to the original input question
             """

prompt_template = ChatPromptTemplate.from_messages(
   [
       ("system", SYS_PROMPT),
       ("human", "Query: {query}"),
       MessagesPlaceholder(variable_name="agent_scratchpad"),
   ]
)

In [49]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

tools = [search_web]
agent = create_tool_calling_agent(chatgpt, tools, prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [50]:
prompt = """Tell me about the recent winners of Australian Open Tennis across Men's and Ladies events.
            Do not guess answers, only answer if you are sure"""
response = agent_executor.invoke({"query": prompt})



> Entering new AgentExecutor chain...

Invoking: `search_web` with `{'query': "2023 Australian Open Men's winner"}`


Searching web and extracting page info


100%|██████████| 5/5 [00:03<00:00,  1.40it/s]


["Australian Open men's singles final score, result as Jannik Sinner cruises past Alexander Zverev for second consecutive title | Sporting News\nSkip to main content\nNFL Scores\nUnited States [English]\nEditions\nUnited States [English]\nArgentina\nAustralia\nBrasil\nCanada\nIndia\n日本\nMéxico\n대한민국\nEspaña\nไทย\nUnited Kingdom\nUnited States [Español]\nViệt Nam\nNFL\nNFL\nHome\nNews\nTeams\nScores\nSchedule\nStandings\nNFL Teams\nAFC East\nBuffalo Bills\nMiami Dolphins\nNew England Patriots\nNew York Jets\nAFC North\nBaltimore Ravens\nCincinnati Bengals\nCleveland Browns\nPittsburgh Steelers\nAFC South\nHouston Texans\nIndianapolis Colts\nJacksonville Jaguars\nTennesse Titans\nAFC West\nDenver Broncos\nKansas City Chiefs\nLas Vegas Raiders\nLos Angeles Chargers\nNFC East\nDallas Cowboys\nNew York Giants\nPhiladelphia Eagles\nWashington Commanders\nNFC North\nChicago Bears\nDetroit Lions\nGreen Bay Packers\nMinnesota Vikings\nNFC South\nAtlanta Falcons\nCarolina Panthers\nNew Orleans S

100%|██████████| 5/5 [00:04<00:00,  1.07it/s]


["Australian Open women's singles final score, result as Madison Keys upsets Aryna Sabalenka for maiden grand slam title | Sporting News\nSkip to main content\nNFL Scores\nUnited States [English]\nEditions\nUnited States [English]\nArgentina\nAustralia\nBrasil\nCanada\nIndia\n日本\nMéxico\n대한민국\nEspaña\nไทย\nUnited Kingdom\nUnited States [Español]\nViệt Nam\nNFL\nNFL\nHome\nNews\nTeams\nScores\nSchedule\nStandings\nNFL Teams\nAFC East\nBuffalo Bills\nMiami Dolphins\nNew England Patriots\nNew York Jets\nAFC North\nBaltimore Ravens\nCincinnati Bengals\nCleveland Browns\nPittsburgh Steelers\nAFC South\nHouston Texans\nIndianapolis Colts\nJacksonville Jaguars\nTennesse Titans\nAFC West\nDenver Broncos\nKansas City Chiefs\nLas Vegas Raiders\nLos Angeles Chargers\nNFC East\nDallas Cowboys\nNew York Giants\nPhiladelphia Eagles\nWashington Commanders\nNFC North\nChicago Bears\nDetroit Lions\nGreen Bay Packers\nMinnesota Vikings\nNFC South\nAtlanta Falcons\nCarolina Panthers\nNew Orleans Saints\n

RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4o in organization org-Q6HT6sFLPnWN5QsRrpaCz3c5 on tokens per min (TPM): Limit 30000, Requested 43779. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}